In [ ]:
from ecallisto_ng.data_fetching.get_data import get_data
from ecallisto_ng.data_processing.utils import elimwrongchannels, subtract_constant_background, subtract_rolling_background
from ecallisto_ng.plotting.utils import fill_missing_timesteps_with_nan
from ecallisto_ng.plotting.plotting import plot_spectogram_mpl, plot_spectogram, plot_with_fixed_resolution_mpl
from ecallisto_ng.data_fetching.get_information import check_table_data_availability, check_min_max_datetime_in_table
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px

### Fetch some data from the e-Callisto API
Because of SQL limtations, the function `ecallisto_ng.data_fetching.get_data.extract_instrument_name` was used to make the instrument names SQL compatible.

In [ ]:
get_ava

In [ ]:
fast_api_parameter =  {
    "instrument_name": "germany_dlr_62",
    "start_datetime": "2023-09-28 00:00:00",
    "end_datetime": "2023-09-28 23:59:00",
    "timebucket": "5min",
    "agg_function": "MAX",
}

In [ ]:
check_min_max_datetime_in_table(**fast_api_parameter)

In [ ]:
# Check first data availability
check_table_data_availability(**fast_api_parameter)

In [ ]:
df = get_data(**fast_api_parameter)

In [ ]:
# Some meta data is available in the returned DataFrame
print(df.attrs)

In [ ]:
df_filled_nan = fill_missing_timesteps_with_nan(df) # To avoid that plotly is interpolating the data during the night

In [ ]:
plot_spectogram(
    df_filled_nan, fast_api_parameter["instrument_name"], fast_api_parameter["start_datetime"], fast_api_parameter["end_datetime"]
)

In [ ]:
plot_spectogram(subtract_constant_background(df_filled_nan), fast_api_parameter["instrument_name"], fast_api_parameter["start_datetime"], fast_api_parameter["end_datetime"])

### Process the data a bit by removing noise, background image and wrong channels

In [ ]:
df_ewc = fill_missing_timesteps_with_nan(df)
df_ewc = elimwrongchannels(df_ewc)

In [ ]:
df_ewc = elimwrongchannels(df)
df_ewc = fill_missing_timesteps_with_nan(df_ewc)

In [ ]:
fig = plot_spectogram(
    df_ewc, fast_api_parameter["instrument_name"], fast_api_parameter["start_datetime"], fast_api_parameter["end_datetime"]
)
fig.show()

In [ ]:
df_ewc_rbg = subtract_rolling_background(df_ewc, window=10)

In [ ]:
fig = plot_spectogram(
    df_ewc_rbg, fast_api_parameter["instrument_name"], fast_api_parameter["start_datetime"], fast_api_parameter["end_datetime"]
)
fig.show()

### Fetch interesting data from the data

In [ ]:
fast_api_parameter =  {
    "instrument_name": "germany_dlr_62",
    "start_datetime": "2023-09-28 09:05:00",
    "end_datetime": "2023-09-28 09:09:00",
}

In [ ]:
df = get_data(**fast_api_parameter)

In [ ]:
df  = fill_missing_timesteps_with_nan(df)

In [ ]:
plot_spectogram(df, fast_api_parameter["instrument_name"], fast_api_parameter["start_datetime"], fast_api_parameter["end_datetime"])

In [ ]:
df_ewc_rbg = subtract_rolling_background(elimwrongchannels(df), window=100, how="quantile", quantile_value=0.05)

In [ ]:
plot_spectogram(df_ewc_rbg, fast_api_parameter["instrument_name"], fast_api_parameter["start_datetime"], fast_api_parameter["end_datetime"])

### Get live data

In [ ]:
from datetime import datetime, timedelta

In [ ]:
now = datetime.now()

fast_api_parameter =  {
    "instrument_name": "austria_unigraz_01",
    "start_datetime": (datetime.now() - timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S"),
    "end_datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "agg_function": "MAX",
    "timebucket": "1min",
}

In [ ]:
df = get_data(**fast_api_parameter)

In [ ]:
plot_spectogram(fill_missing_timesteps_with_nan(df), fast_api_parameter["instrument_name"], fast_api_parameter["start_datetime"], fast_api_parameter["end_datetime"])

In [ ]:
# Pandas piping of course works too
# When using elimwrongchannels, use `fill_missing_timesteps_with_nan` after!
(df.pipe(elimwrongchannels)
        .pipe(subtract_rolling_background, center=False)
        .pipe(fill_missing_timesteps_with_nan)
        .pipe(plot_spectogram, fast_api_parameter["instrument_name"], fast_api_parameter["start_datetime"], fast_api_parameter["end_datetime"])
)

## Non existent data
Ecallisto NG will raise an ValueError in those cases (error handling is still a bit a work in progress.)

In [ ]:
now = datetime.now()

fast_api_parameter =  {
    "instrument_name": "austria_unigraz_01",
    "start_datetime": (datetime.now() - timedelta(hours=24) + timedelta(days=30)).strftime("%Y-%m-%d %H:%M:%S"),
    "end_datetime": (datetime.now() + + timedelta(days=30)).strftime("%Y-%m-%d %H:%M:%S"),
    "agg_function": "MAX",
    "timebucket": "1min",
}

In [ ]:
try:
    get_data(**fast_api_parameter)
except Exception as e:
    print(e)

### Get information of data availability

In [ ]:
from ecallisto_ng.data_fetching.get_information import get_tables, get_table_names_with_data_between_dates
from datetime import datetime, timedelta

In [ ]:
get_tables()

In [ ]:
get_table_names_with_data_between_dates(
    start_datetime=(datetime.now() - timedelta(hours=24)).strftime("%Y-%m-%d %H:%M:%S"),
    end_datetime=datetime.now().strftime("%Y-%m-%d %H:%M:%S")
)

In [ ]:
check_min_max_datetime_in_table(get_tables()[5])

In [ ]:
plot_with_fixed_resolution_mpl('austria_unigraz_01', '2023-03-31 18:46:00', '2022-04-01 18:46:00', resolution=500, sampling_method='MAX')